In [1]:
print("My model of Machine Learning based on K nearest neightbour")

My model of Machine Learning based on K nearest neightbour


In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import pandas as pd
import pymysql
from datetime import datetime, timedelta 
import math

In [2]:
def bubble_sort(nums):
    for i in range(5):
        for j in range(len(nums) - i - 1):  
            if nums[j][0] > nums[j + 1][0]:
                nums[j], nums[j + 1] = nums[j + 1], nums[j]

    return nums

In [3]:
def get_prediction_data(date,time):
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    weekday = datetime.strptime(date,"%Y-%m-%d").weekday()
    #day = datetime.strptime(date,"%Y-%m-%d").day
    time = str(time) + ":00:00"
    dic = get_forecast(date + ' ' + time)
    con = dic_w[dic['condition']]
    dic['condition'] = con
    dic['time'] = date + ' ' + time
    dic['weekday'] = weekday
    return dic

def get_forecast(date):
    day = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    day_now = datetime.now()
    day = (day - day_now).total_seconds()/3600
    hour = int(day / 3)
    url = 'https://api.openweathermap.org/data/2.5/forecast?lat=53.36&lon=-6.25&appid=dbb2b9eb4f9424b9c2c168ad52c077d9'
    info = requests.get(url).json()
    wind_speed = info['list'][hour]['wind']['speed']
    condition = info['list'][hour]['weather'][0]['main']
    return {'wind':wind_speed,'condition':condition}

In [28]:
def get_old_weather(date):
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    date_new = date + timedelta(hours = 8)
    sql = "SELECT * FROM dbike.weather where `datetime` between '"+str(date)+"' and '"+str(date_new)+"';"
    db = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor = db.cursor()
    cursor.execute(sql)
    db.commit()
    data = cursor.fetchall()
    cursor.close()
    db.close()
    #print(data)
    dic_old_weather = {}
    dic_old_weather['condition'] = dic_w[data[0][3]]
    dic_old_weather['wind'] = data[0][9]
    dic_old_weather['temp'] = data[0][6]
    return dic_old_weather

def prediction(number,date,time):
    db0 = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor0 = db0.cursor()
    sql0="SELECT * FROM dbike.daily_data where `number` = " + str(number) + " group by last_update;"
    cursor0.execute(sql0)
    db0.commit()
    data = cursor0.fetchall()
    cursor0.close()
    db0.close()
    length = len(data)
    i = 0
    dic_pre = get_prediction_data(date,time)
    date_new = datetime.strptime(dic_pre['time'],"%Y-%m-%d %H:%M:%S")
    li = []
    low = datetime.strptime('2020-03-04 09:39:43',"%Y-%m-%d %H:%M:%S")
    high = datetime.strptime('2020-03-24 09:47:28',"%Y-%m-%d %H:%M:%S")
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    db = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor = db.cursor()
    dic_old_weather = {}
    dic_old = {}
    while i < length:
        date_old = data[i][4]
        print(date_old)
        datetime.strptime(str(date_old),"%Y-%m-%d %H:%M:%S")
        if date_old < low or date_old > high:
            i = i + 1
            continue
        available_bike_stands = data[i][2]
        available_bikes = data[i][3]
        weekday_old = date_old.weekday()
        
        
        date_new_q = date_old + timedelta(hours = 8)
        print(str(date_old), str(date_new_q))
        cursor.execute("SELECT * FROM dbike.weather where `datetime` between '%s' and '%s' ;" % (str(date_old), str(date_new_q)))
        data_o = cursor.fetchone()
        #print(data_o)
        dic_old_weather['condition'] = dic_w[data_o[3]]
        dic_old_weather['wind'] = data_o[9]
        dic_old_weather['temp'] = data_o[6]
        dic_old = dic_old_weather
        
        
        d_condition = float(dic_pre['condition'])-float(dic_old['condition'])
        d_wind = (float(dic_pre['wind']) - float(dic_old['wind']))*5
        if date_new.hour*10000+date_new.minute*100+date_new.second > date_old.hour*10000+date_old.minute*100+date_old.second:
            d_time = (date_new - date_old).seconds/3600
        else:
            d_time = (date_old - date_new).seconds/3600
        d_week = int(dic_pre['weekday'])-int(weekday_old)
        print(d_condition)
        print(d_wind)
        print(d_time)
        print(d_week)
        distance = math.sqrt(d_wind**2 + d_condition**2 + d_time**2 + d_week**2)
        li.append([distance,available_bikes,available_bike_stands])
        #print(li)
        i = i + 1
    db.commit()
    cursor.close()
    db.close()
    bubble_sort(li)  
    weight = 1/li[0][0] + 1/li[1][0] + 1/li[2][0] + 1/li[3][0] + 1/li[4][0]
    prediction_bikes = (li[0][1]*(1/li[0][0]) + li[1][1]*(1/li[1][0]) + li[2][1]*(1/li[2][0]) + li[3][1]*(1/li[3][0]) + li[4][1]*(1/li[4][0]))/ weight
    prediction_stands = (li[0][2]*(1/li[0][0]) + li[1][2]*(1/li[1][0]) + li[2][2]*(1/li[2][0]) + li[3][2]*(1/li[3][0]) + li[4][2]*(1/li[4][0]))/weight
    dic = {}
    dic['prediction_bikes']=prediction_bikes
    dic['prediction_stands']=prediction_stands
    dic['condition'] = dic_pre['condition']
    dic['wind'] = dic_pre['wind']
    dic['temp'] = dic_pre['temp']
    return dic


In [29]:
print(prediction(32,'2020-04-12','09'))

2020-03-04 09:39:43
2020-03-04 09:39:43 2020-03-04 17:39:43
0.0
2.7500000000000013
0.6619444444444444
4
2020-03-04 09:40:44
2020-03-04 09:40:44 2020-03-04 17:40:44
0.0
1.8500000000000005
0.6788888888888889
4
2020-03-04 10:30:02
2020-03-04 10:30:02 2020-03-04 18:30:02
0.0
1.8500000000000005
1.5005555555555556
4
2020-03-04 14:50:38
2020-03-04 14:50:38 2020-03-04 22:50:38
0.0
1.8500000000000005
5.8438888888888885
4
2020-03-04 14:53:55
2020-03-04 14:53:55 2020-03-04 22:53:55
0.0
1.8500000000000005
5.898611111111111
4
2020-03-04 14:54:18
2020-03-04 14:54:18 2020-03-04 22:54:18
0.0
1.8500000000000005
5.905
4
2020-03-04 14:58:57
2020-03-04 14:58:57 2020-03-04 22:58:57
0.0
1.8500000000000005
5.9825
4
2020-03-04 15:07:39
2020-03-04 15:07:39 2020-03-04 23:07:39
0.0
1.8500000000000005
6.1275
4
2020-03-04 15:11:43
2020-03-04 15:11:43 2020-03-04 23:11:43
0.0
-3.1499999999999995
6.195277777777778
4
2020-03-04 15:18:07
2020-03-04 15:18:07 2020-03-04 23:18:07
0.0
-3.1499999999999995
6.301944444444445


KeyboardInterrupt: 

In [ ]:
'''
t1 = '2020-03-04 21:18:29'
t2 = '2020-03-06 23:25:59'
t1 = datetime.strptime(t1,"%Y-%m-%d %H:%M:%S")
t2 = datetime.strptime(t2,"%Y-%m-%d %H:%M:%S")
t = t2-t1
t = t.seconds/1800
print(t)
t = ((t2-t1).total_seconds())/(3600*24)
t2 = '2020-04-11'
t2 = datetime.strptime(t2,"%Y-%m-%d").day
t = datetime.now().day
print(t)
date = '2020-04-14 21:18:29'
day = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
day_now = datetime.now()
day = (day - day_now).total_seconds()/3600
print(day/3)
'''